# Packages, dependencies, extras

In [59]:
#packages
from IPython.display import clear_output
#!pip install schedule

from config import rh_username,rh_password
#from my_config import rh_username,rh_password
import pandas_datareader.data as web
from datetime import datetime
import robin_stocks as r
import schedule,time
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

clear_output()
pd.set_option('display.max_rows', None)

In [100]:
#Add color for a bit of class.
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
 
#USAGE
# f"{bcolors.OKGREEN}STRING\n{bcolors.ENDC}"

# Robinhood login & functionality

In [ ]:
#https://robin-stocks.readthedocs.io/en/latest/
login = r.authentication.login(username=rh_username, 
                                  password=rh_password, 
                                  #expiresIn=86400, 
                                  #scope='internal', 
                                  #by_sms=True, 
                                  store_session=True, 
                                  #mfa_code=None
                               )
access_token=login['access_token']
token_type=login['token_type']

In [60]:
#Crypto functions for reference.
"""Contains functions to get information about crypto-currencies."""
import robin_stocks.helper as helper
import robin_stocks.urls as urls

@helper.login_required
def load_crypto_profile(info=None):
    """Gets the information associated with the crypto account.
    :param info: The name of the key whose value is to be returned from the function.
    :type info: Optional[str]
    :returns: [dict] The function returns a dictionary of key/value pairs. \
    If a string is passed in to the info parameter, then the function will return \
    a string corresponding to the value of the key whose name matches the info parameter.
    :Dictionary Keys: * apex_account_number
                      * created_at
                      * id
                      * rhs_account_number
                      * status
                      * status_reason_code
                      * updated_at
                      * user_id
    """
    url = urls.crypto_account()
    data = helper.request_get(url, 'indexzero')
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_positions(info=None):
    """Returns crypto positions for the account.
    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: [list] Returns a list of dictionaries of key/value pairs for each option. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.
    :Dictionary Keys: * account_id
                      * cost_basis
                      * created_at
                      * currency
                      * id
                      * quantity
                      * quantity_available
                      * quantity_held_for_buy
                      * quantity_held_for_sell
                      * updated_at
    """
    url = urls.crypto_holdings()
    data = helper.request_get(url, 'pagination')
    return(helper.filter_data(data, info))


def get_crypto_currency_pairs(info=None):
    """Gets a list of all the cypto currencies that you can trade.
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [list] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * asset_currency
                      * display_only
                      * id
                      * max_order_size
                      * min_order_size
                      * min_order_price_increment
                      * min_order_quantity_increment
                      * name
                      * quote_currency
                      * symbol
                      * tradability
    """
    url = urls.crypto_currency_pairs()
    data = helper.request_get(url, 'results')
    return(helper.filter_data(data, info))


def get_crypto_info(symbol, info=None):
    """Gets information about a crpyto currency.
    :param symbol: The crypto ticker.
    :type symbol: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then will return a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a strings representing the value of the key.
    :Dictionary Keys: * asset_currency
                      * display_only
                      * id
                      * max_order_size
                      * min_order_size
                      * min_order_price_increment
                      * min_order_quantity_increment
                      * name
                      * quote_currency
                      * symbol
                      * tradability
    """
    url = urls.crypto_currency_pairs()
    data = helper.request_get(url, 'results')
    data = [x for x in data if x['asset_currency']['code'] == symbol]
    if len(data) > 0:
        data = data[0]
    else:
        data = None
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_quote(symbol, info=None):
    """Gets information about a crypto including low price, high price, and open price
    :param symbol: The crypto ticker.
    :type symbol: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * ask_price
                      * bid_price
                      * high_price
                      * id
                      * low_price
                      * mark_price
                      * open_price
                      * symbol
                      * volume
 
    """
    id = get_crypto_info(symbol, info='id')
    url = urls.crypto_quote(id)
    data = helper.request_get(url)
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_quote_from_id(id, info=None):
    """Gets information about a crypto including low price, high price, and open price. Uses the id instead of crypto ticker.
    :param id: The id of a crypto.
    :type id: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * ask_price
                      * bid_price
                      * high_price
                      * id
                      * low_price
                      * mark_price
                      * open_price
                      * symbol
                      * volume
    """
    url = urls.crypto_quote(id)
    data = helper.request_get(url)
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_historicals(symbol, interval='hour', span='week', bounds='24_7', info=None):
    """Gets historical information about a crypto including open price, close price, high price, and low price.
    :param symbol: The crypto ticker.
    :type symbol: str
    :param interval: The time between data points. Can be '15second', '5minute', '10minute', 'hour', 'day', or 'week'. Default is 'hour'.
    :type interval: str
    :param span: The entire time frame to collect data points. Can be 'hour', 'day', 'week', 'month', '3month', 'year', or '5year'. Default is 'week'
    :type span: str
    :param bound: The times of day to collect data points. 'Regular' is 6 hours a day, 'trading' is 9 hours a day, \
    'extended' is 16 hours a day, '24_7' is 24 hours a day. Default is '24_7'
    :type bound: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [list] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * begins_at
                      * open_price
                      * close_price
                      * high_price
                      * low_price
                      * volume
                      * session
                      * interpolated
                      * symbol
    """
    interval_check = ['15second', '5minute', '10minute', 'hour', 'day', 'week']
    span_check = ['hour', 'day', 'week', 'month', '3month', 'year', '5year']
    bounds_check = ['24_7', 'extended', 'regular', 'trading']

    if interval not in interval_check:
        print(
            'ERROR: Interval must be "15second","5minute","10minute","hour","day",or "week"', file=helper.get_output())
        return([None])
    if span not in span_check:
        print('ERROR: Span must be "hour","day","week","month","3month","year",or "5year"', file=helper.get_output())
        return([None])
    if bounds not in bounds_check:
        print('ERROR: Bounds must be "24_7","extended","regular",or "trading"', file=helper.get_output())
        return([None])
    if (bounds == 'extended' or bounds == 'trading') and span != 'day':
        print('ERROR: extended and trading bounds can only be used with a span of "day"', file=helper.get_output())
        return([None])


    symbol = helper.inputs_to_set(symbol)
    id = get_crypto_info(symbol[0], info='id')
    url = urls.crypto_historical(id)
    payload = {'interval': interval,
               'span': span,
               'bounds': bounds}
    data = helper.request_get(url, 'regular', payload)

    histData = []
    cryptoSymbol = data['symbol']
    for subitem in data['data_points']:
        subitem['symbol'] = cryptoSymbol
        histData.append(subitem)

    return(helper.filter_data(histData, info))

# SuperTrend Formula

In [62]:
def tr(data):
    data['previous_close'] = data['close_price'].shift(1)
    data['high-low'] = abs(data['high_price'] - data['low_price'])
    data['high-pc'] = abs(data['high_price'] - data['previous_close'])
    data['low-pc'] = abs(data['low_price'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high_price'] + df['low_price']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close_price'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close_price'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

# Robinhood data

In [10]:
#Live rh data
get_crypto_quote('DOGE')

{'ask_price': '0.277982',
 'bid_price': '0.276493',
 'mark_price': '0.277237',
 'high_price': '0.281927',
 'low_price': '0.269476',
 'open_price': '0.274902',
 'symbol': 'DOGEUSD',
 'id': '1ef78e1b-049b-4f12-90e5-555dcf2fe204',
 'volume': '0.000000'}

In [129]:
#Retrieve DOGE & BTC data
symbol='DOGE'
crypto_historical=pd.DataFrame.from_dict(r.crypto.get_crypto_historicals(symbol, interval='hour', span='day', bounds='24_7', info=None))
crypto_historical['begins_at']=pd.to_datetime(crypto_historical['begins_at'], format="%Y-%m-%dT%H:%M:%fZ")

crypto_historical['close_price'] = crypto_historical.apply(lambda x : float(x['close_price']),axis=1)
doge=crypto_historical.drop(columns=['session','interpolated','symbol','volume']).apply(pd.to_numeric)
doge['begins_at'] = doge.apply(lambda x : pd.Timestamp(x['begins_at'],),axis=1)

symbol='BTC'
crypto_historical=pd.DataFrame.from_dict(r.crypto.get_crypto_historicals(symbol, interval='hour', span='day', bounds='24_7', info=None))
crypto_historical['begins_at']=pd.to_datetime(crypto_historical['begins_at'], format="%Y-%m-%dT%H:%M:%fZ")

crypto_historical['close_price'] = crypto_historical.apply(lambda x : float(x['close_price']),axis=1)
btc=crypto_historical.drop(columns=['session','interpolated','symbol','volume']).apply(pd.to_numeric)
btc['begins_at'] = btc.apply(lambda x : pd.Timestamp(x['begins_at'],),axis=1)

# SuperTrend application

In [137]:
#Checking for signals
in_position = False

def check_buy_sell_signals(df):
    global in_position

    print(f"{bcolors.OKCYAN}\nchecking for buy/sell signals{bcolors.ENDC}")
    #print(df.tail(1))
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print(f"{bcolors.OKGREEN}changed to uptrend, buy\n{bcolors.ENDC}")
        if not in_position:
            print(order)
            in_position = True
        else:
            print("already in position, nothing to do\n")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print(f"{bcolors.WARNING}changed to downtrend, sell\n{bcolors.ENDC}")
            print(order)
            in_position = False
        else:
            print("You aren't in position, nothing to sell\n")

def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    bars = doge
    df = pd.DataFrame(bars[:-1], columns=['begins_at', 'open_price', 'high_price', 'low_price', 'close_price'])
    df['begins_at'] = pd.to_datetime(df['begins_at'], unit='ms')

    supertrend_data = supertrend(df)
    
    check_buy_sell_signals(supertrend_data)

In [110]:
df.tail(5)

,open_price,close_price,high_price,low_price,previous_close,high-low,high-pc,low-pc,tr
begins_at,,,,,,,,,
2021-08-31T11:00:00Z,0.275274,0.279601,0.280640,0.273985,0.275274,0.006655,0.005366,0.001289,0.006655
2021-08-31T12:00:00Z,0.279601,0.277411,0.281031,0.275757,0.279601,0.005274,0.001430,0.003844,0.005274
2021-08-31T13:00:00Z,0.277902,0.279449,0.280695,0.276929,0.277411,0.003766,0.003284,0.000482,0.003766
2021-08-31T14:00:00Z,0.279449,0.275362,0.281927,0.274335,0.279449,0.007592,0.002478,0.005114,0.007592
2021-08-31T15:00:00Z,0.275362,0.278389,0.279644,0.273835,0.275362,0.005809,0.004282,0.001527,0.005809


In [138]:
#Execute practice bot
schedule.every(10).seconds.do(run_bot)

while True:
    schedule.run_pending()
    time.sleep(1)

Fetching new bars for 2021-08-31T09:43:50.573182

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.603115

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.624691

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.647346

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.660193

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.682092

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.711132

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.737942

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.759217

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.775252

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.797478

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:50.811612

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:43:5

Fetching new bars for 2021-08-31T09:44:11.648549

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:11.666456

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:11.691599

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.712311

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.741677

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.760137

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.781122

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.800938

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.810306

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.819318

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.833486

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:12.852595

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:1

Fetching new bars for 2021-08-31T09:44:33.600133

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:33.617511

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:33.640655

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.658510

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.688397

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.709341

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.728465

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.747911

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.756435

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.764817

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.776718

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:34.796987

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:3

Fetching new bars for 2021-08-31T09:44:54.421565

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.452801

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.468657

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.495327

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.509514

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.520389

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:54.536105

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:55.552048

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:55.569846

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:55.597487

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:56.617621

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:56.649747

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:44:5

Fetching new bars for 2021-08-31T09:45:15.255960

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:15.276886

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.307194

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.336577

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.357732

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.374972

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.397432

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.407725

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:16.416965

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:17.434007

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:17.457667

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:17.475599

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:45:1

...

Fetching new bars for 2021-08-31T09:51:21.705309

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:21.723174

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.752338

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.768945

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.791606

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.811389

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.823282

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.841977

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:22.855469

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:23.868395

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:23.898269

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:23.915459

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:2

Fetching new bars for 2021-08-31T09:51:43.654320

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:43.673518

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.699675

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.716081

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.738260

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.759028

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.773038

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.789529

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:44.806134

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:45.821489

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:45.853133

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:45.869547

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:51:4

Fetching new bars for 2021-08-31T09:52:04.461661

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.474711

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.495460

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.511005

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.524046

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.546257

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:04.563051

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:05.578949

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:05.608023

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:06.628872

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:06.656823

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:06.673969

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:0

Fetching new bars for 2021-08-31T09:52:25.353445

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:25.385974

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.410887

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.436532

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.461265

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.478071

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.499956

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.520161

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:26.551786

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:27.577327

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:27.605999

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:28.625099

checking for buy/sell signals
Fetching new bars for 2021-08-31T09:52:2

KeyboardInterrupt: 

In [ ]:
#En Fin